In [360]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from collections import defaultdict
from random import sample
from tqdm.notebook import tqdm

In [387]:
deal_observation = pd.read_csv('dealobservation.csv')
deal_scan = pd.read_csv('dealscan.csv', encoding='latin-1')

/Users/robert1003/.pyenv/versions/3.8.5/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (22,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [362]:
print('deal_observation', deal_observation.columns, sep='\n')
print('deal_scan', deal_scan.columns, sep='\n')

deal_observation
Index(['DateAnnounced', 'DateEffective', 'DateWithdrawn', 'TargetName',
       'TargetPrimarySICCode', 'tcountry', 'AcquirorName',
       'AcquirorPrimarySICCode', 'acountry_sub', 'Status', 'ofSharesAcq',
       'OwnedAfterTransaction', 'sought', 'ValueofTransactionmil',
       'TargetTotalFeesmil', 'AcquirorTotalFeesmil',
       'TargetandAcquirorFinancialAd', 'AcquirorDatastreamCode',
       'TargetDatastreamCode', 'OfferPricetoTargetStockPric', 'U', 'V',
       'TargetPublicStatus', 'AcqPublicStatus', 'Form', 'AcquirorCUSIP',
       'TargetCUSIP', 'Attitude', 'Acquirorsterminationfeemil',
       'Targetsterminationfeemil', 'AcquirorMarketVal4WeeksPrio',
       'MarketPriceBookValue', 'TgtMarketVal4WwksPriorto',
       'AcquirorCashLTMmil', 'TargetLockup', 'AcquirorTotalAssetsmil',
       'acountry_parent', 'BuysideGovernmentOwnedInvolve',
       'SellsideGovernmentOwnedInvolv', 'ofCash', 'ofStock', 'AP',
       'acq_nation_sub', 'ascow_sub', 'target_nation', 'tscow'

In [363]:
deal_scan[['company', 'country', 'bcompany', 'bcountry']].head()

,company,country,bcompany,bcountry
0,National Australia Bank Ltd [NAB],Australia,*BGL,Other
1,Airservices Australia,Australia,*BGL,Other
2,Hommell,France,*BUMF,France
3,Iran Petrochemical Commercial Co [IPCC],Iran,*Bank Saderat Iran,Iran
4,Seche Environnement SA,France,*Banque Espirito Santo et de la Venetie [BESV],France


In [364]:
deal_observation[['AcquirorName', 'acountry_sub', 'TargetName', 'tcountry']].head()
# Same CUSIP may have different company name. Some is caused by company changing name, others I don't know

,AcquirorName,acountry_sub,TargetName,tcountry
0,Ariana(Afghanistan),Afghanistan,Air France-3 Boeing 727 Aircft,France
1,Cevital SA,Algeria,Fagorbrandt SAS,France
2,Pluspetrol Angola Corp,Angola,Cabinda Onshore South Block,Angola
3,Banco CMF SA,Argentina,Eurobanco Bank Ltd,Bahamas
4,Moinho Fluminense SA,Brazil,Hershey do Brasil,Brazil


### normalize name

In [342]:
from cleanco import basename
import string

def normalize(name):
    if name.endswith(']'):
        if name.find('[') != -1:
            name = name[:len(name)-name[::-1].index('[')]
        elif name.find('{') != -1:
            name = name[:len(name)-name[::-1].index('{')]
        else:
            print(name)
            assert False
    name = name.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    
    while name != basename(name):
        name = basename(name)
    
    name = ' '.join(map(lambda x: x.lower(), name.split()))
    return name

# [(id, (name, country))...]
def merge_namelist(namelist):
    cache = {}
    def query(name):
        if name not in cache:
            cache[name] = normalize(name)
        return cache[name]
    
    pa = {}
    size = {}
    def find(x):
        if pa[x] != x:
            pa[x] = find(pa[x])
        return pa[x]
    def merge(x, y):
        x, y = find(x), find(y)
        if x == y:
            return
        if size[x] < size[y]:
            x, y = y, x
        pa[y] = x
        size[x] += size[y]
        size[y] = 0
    
    newgroup = defaultdict(lambda: list())
    for o in tqdm(namelist):
        cname = query(o[1][0])
        ccountry = o[1][1] if pd.isna(o[1][1]) else query(o[1][1])
        pa[o[0]] = o[0]
        size[o[0]] = 1
        newgroup[cname[:3]].append((o[0], (cname, ccountry)))
    
    for g in tqdm(newgroup):
        for i in tqdm(newgroup[g], leave=False):
            score = []
            for j in newgroup[g]:
                if i[0] == j[0]:
                    continue
                if (i[1][1] != j[1][1]) and (not pd.isna(i[1][1]) or not pd.isna(j[1][1])):
                    continue
                score.append((j[0], fuzz.ratio(i[1][0], j[1][0])))
                if score[-1][1] == 100:
                    break
            score = sorted(score, key=lambda x: x[1], reverse=True)
            if len(score) > 0 and score[0][1] >= 95:
                merge(i[0], score[0][0])
    
    newid = {}
    for i in namelist:
        newid[i[0]] = find(i[0])
        
    return newid

In [325]:
company_set = set(zip(deal_scan['company'], deal_scan['country']))
company_set.update(zip(deal_scan['bcompany'], deal_scan['bcountry']))
company_set.update(zip(deal_observation['AcquirorName'], deal_observation['acountry_sub']))
company_set.update(zip(deal_observation['TargetName'], deal_observation['tcountry']))

In [326]:
company_list = [(i, j) for i, j in enumerate(company_set)]

In [343]:
newid_func = merge_namelist(company_list)

In [376]:
new_company_list = [(newid_func[i[0]], i[1]) for i in company_list]
rev_new_company_list = {i[1]:i[0] for i in new_company_list}

In [358]:
group = defaultdict(lambda: [])
for i in newid_func:
    group[newid_func[i]].append(i)

cnt = 0
for _, g in group.items():
    if len(g) > 1:
        print(g)
        print(list(map(lambda x: company_list[x][1][0], g)))
        cnt += 1

[26, 155306]
['Simple Health & Beauty Group', 'Simple Health & Beauty Group Ltd [ex-Accantia Health & Beauty Ltd]']
[44, 134229]
['Greenstone Farm Credit Services ACA/FLCA', 'Greenstone Farm Credit Services FLCA']
[46, 69445]
['Pohjola Bank Plc', 'Pohjola Bank Plc [Ex-OKO Bank Plc]']
[159, 41342]
['Vodacom Group Ltd', 'Vodacom Group(Pty)Ltd']
[166, 35038]
['CD Bramall Plc', 'CD Bramall PLC']
[167, 119364]
['Host Hotels & Resorts Inc', 'Host Hotels & Resorts LP']
[182, 131436]
['Scitex', 'Scitex Corp Ltd']
[215, 80021]
['NationsBanc Capital Markets', 'NationsBank Capital Markets']
[258, 20017]
['Long-Term Credit Bank of Japan Ltd', 'Long Term Credit Bank of Japan']
[282, 57937]
['Valiant Petroleum PLC', 'Valiant Petroleum Plc']
[293, 18697]
['Phadia AB [Ex-Pharmacia Diagnostics AB]', 'Phadia AB']
[304, 6061]
['LaSalle Income & Growth Fund IV', 'LaSalle Income & Growth Fund V']
[319, 25718]
['NM Rothschild & Sons Ltd', 'NM Rothschild & Sons Ltd [Manchester]']
[332, 86663]
['Carpetright P

['DaimlerChrysler North America Holdings [ex Daimler-Benz North America]', 'DaimlerChrysler North America Holding Corp', 'DaimlerChrysler North America Holdings [ex-DaimlerChrysler NA]']
[22492, 128545]
['Shiroki Inc', 'Shiroki Corp']
[22506, 66230]
['Tiphook Plc', 'Tiphook PLC']
[22511, 36478]
['Jarvis Plc', 'Jarvis PLC']
[22518, 64345]
['Silver Fern Farms Ltd [Ex-PPCS Ltd]', 'Silver Fern Farms Ltd']
[22600, 88013, 140684]
['URSA Bank JSC', 'URSA Bank JSC [Ex-Sibacadembank OJSC]', 'URSA Bank JSC [Ex-UralVneshtorgbank]']
[22612, 118299]
['Banksia Series 1 Trust', 'Banksia Series 2 Trust']
[22653, 78798]
['Fresenius Medical Care AG & KGaA [FMC]', 'Fresenius Medical Care AG & Co']
[22655, 42960]
['BAA PLC', 'BAA Ltd']
[22670, 136798, 147113, 147610]
['DVB Bank AG [Ex-Deutsche Verkehrs-Bank AG]', 'DVB Bank AG [Ex-Deutsche Verkehrs-Kredit-Bank]', 'DVB Bank AG [Ex-Deutsche VerkehrsBank AG]', 'DVB Bank AG']
[22702, 155256]
['Singapore Technologies Pte Ltd', 'Singapore Technologies']
[22705, 

['Friends Provident Plc', 'Friends Provident PLC']
[44373, 145932]
['Holmes Place PLC', 'Holmes Place Plc']
[44374, 56927, 104368, 144250]
['E.ON AG [EON]', 'E.ON AG [Ex-Veba AG]', 'E ON SE', 'E ON AG']
[44418, 136289]
['Pascual Hermanos SL', 'Pascual Hermanos SA']
[44452, 60212]
['Kleinwort Benson', 'Kleinwort Benson Ltd']
[44460, 52283]
['St Ives PLC', 'St Ives Plc']
[44474, 58481, 79974, 119822]
['DNB', 'DNB ASA [ex-Den Norske Bank]', 'DNB ASA', 'DNB ASA [ex-DnB NOR Bank]']
[44495, 92573]
['Tecnomatix Technologies Ltd', 'Technomatix Technologies Ltd']
[44533, 143028]
['KKR Financial CLO 2005-1', 'KKR Financial CLO 2005-2']
[44543, 59159]
['Zambian Breweries Plc', 'Zambian Breweries PLC']
[44560, 51738]
['Inversiones CMPC SA', 'Inversiones CMPC']
[44576, 123256]
['Minorco Finance (UK) PLC', 'Minorco Finance']
[44590, 93213]
['Dollond & Aitchison Group Plc', 'Dollond & Aitchison Group PLC']
[44593, 127517]
['Corporate Services Group PLC', 'Corporate Services Group Plc']
[44597, 108741

[71187, 90781]
['Kexim Bank (UK) Ltd', 'Kexim Bank (UK) Ltd [Ex Export-Import Bank of Korea London]']
[71213, 156593]
['Maruko Co Ltd', 'Maruko Inc']
[71335, 90112]
['William Charles Ltd', 'Williams Charles LTD']
[71340, 105526]
['Jackson National Life Ins Co', 'Jackson National Life Ins Inc']
[71353, 78112]
['Porteous Fastener Co', 'Porteus Fastener Co']
[71389, 137401]
['VTB Bank Europe Plc', 'VTB Bank Europe Plc [Ex-Moscow Narodny Bank Ltd]']
[71395, 158320]
['Beazley Group PLC', 'Beazley Group Plc']
[71407, 151714]
['Jimmy Choo Ltd', 'Jimmy Choo PLC']
[71413, 144279]
['Transgarant OOO', 'Transgrant']
[71516, 83186, 91082, 142323, 158620]
['Carlyle High Yield Partners II Ltd', 'Carlyle High Yield Partners VI', 'Carlyle High Yield Partners III Ltd', 'Carlyle High Yield Partners IV Ltd', 'Carlyle High Yield Partners']
[71581, 103755]
['Comunidad Autonoma de la Region de Murcia [CARM]', 'Comunidad Autonoma de la Region de Murcia [ex-Comunidad de Murcia]']
[71614, 73165]
['Siam United S

[116589, 144531]
['Banco de Credito Inversiones SA [BCI]', 'Banco de Credito Inversiones']
[116703, 141257]
['North Westerly CLO II BV', 'North Westerley CLO I BV']
[116722, 151862]
['Sears Credit II', 'Sears Credit I']
[116839, 158608]
['Frontier Kemper Constructions', 'Frontier-Kemper Constructors']
[116913, 117536]
['Gulf Industrial Investment Co', 'Gulf Industrial Investment Co [GIIC]']
[117068, 118981, 119346]
['Aviva PLC', 'Aviva Plc [ex-CGNU Group]', 'Aviva Plc']
[117072, 151724]
['Undisclosed Oil & Gas Co', 'Undisclosed Oil & Gas']
[117102, 135272]
['Hansol Paper Co Ltd', 'Han Sol Paper Co Ltd']
[117226, 157907]
['Evenflo & Co', 'Evenflo Co Inc [Evenflo & Co]']
[117317, 122179]
['FI Group', 'FI Group PLC']
[117712, 157787]
['Palabora Mining Co Ltd', 'Palabora Mining Co']
[117713, 158541]
['Continental Engineering Co', 'Continental Engineering Corp']
[117825, 124192, 125864, 151152, 157239]
['Korea First Mortgage No 6 Ltd', 'Korea First Mortgage No 3 Plc', 'Korea First Mortgage 

In [381]:
rev_new_company_list[('ProLogis MX Fund Sub LP', 'USA')]

1232

In [375]:
deal_scan.iloc[0]['company']

'National Australia Bank Ltd [NAB]'

In [394]:
def f(x):
    company_name = x.iloc[0]['company']
    country_name = x.iloc[0]['country']
    x['_company_id'] = rev_new_company_list[(company_name, country_name)]
    return x
deal_scan = deal_scan.groupby(['company', 'country']).apply(f)

In [395]:
def f(x):
    company_name = x.iloc[0]['bcompany']
    country_name = x.iloc[0]['bcountry']
    x['_bcompany_id'] = rev_new_company_list[(company_name, country_name)]
    return x
deal_scan = deal_scan.groupby(['bcompany', 'bcountry']).apply(f)

In [396]:
def f(x):
    company_name = x.iloc[0]['AcquirorName']
    country_name = x.iloc[0]['acountry_sub']
    x['_acquiror_id'] = rev_new_company_list[(company_name, country_name)]
    return x
deal_observation = deal_observation.groupby(['AcquirorName', 'acountry_sub']).apply(f)

In [397]:
def f(x):
    company_name = x.iloc[0]['TargetName']
    country_name = x.iloc[0]['tcountry']
    x['_target_id'] = rev_new_company_list[(company_name, country_name)]
    return x
deal_observation = deal_observation.groupby(['TargetName', 'tcountry']).apply(f)

In [403]:
deal_scan.to_csv('dealscan_with_id.csv', index=False)
deal_observation.to_csv('dealobservation_with_id.csv', index=False)